# SSDF master catalogue
## Preparation of DES data

Blanco DES catalogue: the catalogue comes from `dmu0_DES`.

In the catalogue, we keep:

- The identifier (it's unique in the catalogue);
- The position;
- The G band stellarity;
- The magnitude for each band.
- The auto/kron magnitudes/fluxes to be used as total magnitude.
- The PSF fitted madnitudes/fluxes are used as aperture magnitudes.

We don't know when the maps have been observed. We will take the final observation date as 2017.

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from collections import OrderedDict
import os

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, mag_to_flux

In [ ]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL = "des_ra"
DEC_COL = "des_dec"

## I - Column selection

In [ ]:

imported_columns = OrderedDict({
        'COADD_OBJECT_ID': "des_id",
        'RA': "des_ra",
        'DEC': "des_dec",
        'CLASS_STAR_G':  "des_stellarity",
        'FLUX_AUTO_G': "f_decam_g", 
        'FLUXERR_AUTO_G': "ferr_decam_g", 
        'WAVG_FLUX_PSF_G': "f_ap_decam_g", 
        'WAVG_FLUXERR_PSF_G': "ferr_ap_decam_g",
        'MAG_AUTO_G': "m_decam_g", 
        'MAGERR_AUTO_G': "merr_decam_g", 
        'WAVG_MAG_PSF_G': "m_ap_decam_g", 
        'WAVG_MAGERR_PSF_G': "merr_ap_decam_g",
    
        'FLUXERR_AUTO_R': "ferr_decam_r", 
        'WAVG_FLUX_PSF_R': "f_ap_decam_r", 
        'WAVG_FLUXERR_PSF_R': "ferr_ap_decam_r",
        'MAG_AUTO_R': "m_decam_r", 
        'MAGERR_AUTO_R': "merr_decam_r", 
        'WAVG_MAG_PSF_R': "m_ap_decam_r", 
        'WAVG_MAGERR_PSF_R': "merr_ap_decam_r",
    
        'FLUXERR_AUTO_I': "ferr_decam_i", 
        'WAVG_FLUX_PSF_I': "f_ap_decam_i", 
        'WAVG_FLUXERR_PSF_I': "ferr_ap_decam_i",
        'MAG_AUTO_I': "m_decam_i", 
        'MAGERR_AUTO_I': "merr_decam_i", 
        'WAVG_MAG_PSF_I': "m_ap_decam_i", 
        'WAVG_MAGERR_PSF_I': "merr_ap_decam_i",
    
        'FLUXERR_AUTO_Z': "ferr_decam_z", 
        'WAVG_FLUX_PSF_Z': "f_ap_decam_z", 
        'WAVG_FLUXERR_PSF_Z': "ferr_ap_decam_z",
        'MAG_AUTO_Z': "m_decam_z", 
        'MAGERR_AUTO_Z': "merr_decam_z", 
        'WAVG_MAG_PSF_Z': "m_ap_decam_z", 
        'WAVG_MAGERR_PSF_Z': "merr_ap_decam_z",
    
        'FLUXERR_AUTO_Y': "ferr_decam_y", 
        'WAVG_FLUX_PSF_Y': "f_ap_decam_y", 
        'WAVG_FLUXERR_PSF_Y': "ferr_ap_decam_y",
        'MAG_AUTO_Y': "m_decam_y", 
        'MAGERR_AUTO_Y': "merr_decam_y", 
        'WAVG_MAG_PSF_Y': "m_ap_decam_y", 
        'WAVG_MAGERR_PSF_Y': "merr_ap_decam_y",

    })


catalogue = Table.read("../../dmu0/dmu0_DES/data/DES-DR1_SSDF.fits")[list(imported_columns)]
for column in imported_columns:
    catalogue[column].name = imported_columns[column]

epoch = 2017

# Clean table metadata
catalogue.meta = None

In [ ]:
# Adding flux and band-flag columns
for col in catalogue.colnames:
    if col.startswith('m_'):
        
        errcol = "merr{}".format(col[1:])
        f_col = "f{}".format(col[1:])
        f_errcol = "ferr{}".format(col[1:])
        
        # Some object have a magnitude to 99., we suppose this means a missing value
        mask =(catalogue[col] > 90. )
        catalogue[col][mask] = np.nan
        catalogue[errcol][mask] = np.nan
 

        
        # Band-flag column
        if "ap" not in col:
            catalogue.add_column(Column(np.zeros(len(catalogue), dtype=bool), name="flag{}".format(col[1:])))
            
    if col.startswith('f_'):
        
        errcol = "ferr{}".format(col[1:])
        # Some objects have -99.0 values
        mask = (np.isclose(catalogue[col], -99.) )
        catalogue[col][mask] = np.nan
        catalogue[errcol][mask] = np.nan
            
            
        


In [ ]:
catalogue[:10].show_in_notebook()

## II - Removal of duplicated sources

We remove duplicated objects from the input catalogues.

In [ ]:
SORT_COLS = ['merr_ap_decam_g', 'merr_ap_decam_r','merr_ap_decam_i','merr_ap_decam_z','merr_ap_decam_y']
FLAG_NAME = 'des_flag_cleaned'

nb_orig_sources = len(catalogue)

catalogue = remove_duplicates(catalogue, RA_COL, DEC_COL, sort_col=SORT_COLS,flag_name=FLAG_NAME)

nb_sources = len(catalogue)

print("The initial catalogue had {} sources.".format(nb_orig_sources))
print("The cleaned catalogue has {} sources ({} removed).".format(nb_sources, nb_orig_sources - nb_sources))
print("The cleaned catalogue has {} sources flagged as having been cleaned".format(np.sum(catalogue[FLAG_NAME])))

## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [ ]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_SSDF.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec, near_ra0=True)

In [ ]:
delta_ra, delta_dec =  astrometric_correction(
    SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]),
    gaia_coords
)

print("RA correction: {}".format(delta_ra))
print("Dec correction: {}".format(delta_dec))

In [ ]:
catalogue[RA_COL] +=  delta_ra.to(u.deg)
catalogue[DEC_COL] += delta_dec.to(u.deg)

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec, near_ra0=True)

## IV - Flagging Gaia objects

In [ ]:
catalogue.add_column(
    gaia_flag_column(SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]), epoch, gaia)
)

In [ ]:
GAIA_FLAG_NAME = "des_flag_gaia"

catalogue['flag_gaia'].name = GAIA_FLAG_NAME
print("{} sources flagged.".format(np.sum(catalogue[GAIA_FLAG_NAME] > 0)))

## V - Flagging objects near bright stars

# VI - Saving to disk

In [ ]:
catalogue.write("{}/DES.fits".format(OUT_DIR), overwrite=True)